In [0]:
import os, sys
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob

In [3]:
!pip3 install q keras==2.0.5

     |████████████████████████████████| 225kB 12.9MB/s 
  Created wheel for keras: filename=Keras-2.0.5-cp36-none-any.whl size=253392 sha256=4aa7c06fecf2658d6ea4f2674aadbeb6394af55319450d7b5cce34367f097694
  Stored in directory: /root/.cache/pip/wheels/ee/09/df/2f10a3b36c903a620183396e24452e7de4dba01861b1d87cea
Successfully built keras
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.0.5 which is incompatible.
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


In [4]:
!pip3 install tensorflow==1.13.2

     |████████████████████████████████| 92.6MB 104kB/s 
     |████████████████████████████████| 368kB 47.7MB/s 
     |████████████████████████████████| 3.2MB 50.5MB/s 
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0
  Found existing installation: tensorflow 2.2.0rc2
    Uninstalling tensorflow-2.2.0rc2:
      Successfully uninstalled tensorflow-2.2.0rc2


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import cv2
import pylab as pla
import re
import tensorflow as tf
import keras.backend.tensorflow_backend as ktf
from scipy.stats import norm
from keras.layers import Input, Dense, Lambda, Conv2D, MaxPooling2D, Deconv2D, UpSampling2D, Deconvolution2D, Flatten, ZeroPadding2D, merge
from keras.models import Sequential
from keras.models import Model
from keras.regularizers import l2
from keras import backend as K
from keras import objectives
from keras.datasets import mnist
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras import callbacks

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

Using TensorFlow backend.


In [7]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
session = tf.Session(config=config)
ktf.set_session(session)

batch_size = 16
h = 240
w = 320
channel = 3
m_ori = 6
n = 30 * 40 * 8
dim = 80 * 60 * 3
hidden_h = 30
hidden_w = 40
hidden_c = 8
hidden_dim1 = 256
hidden_dim = 128
epsilon_std = 1.0
use_loss = 'mse' # 'mse' or 'xent'
decay = 1e-4 # weight decay, a.k. l2 regularization
use_bias = True


np.random.seed(1111)  

############################################# original network #######################################
## Encoder
#import pdb;pdb.set_trace()
leakyrelu = LeakyReLU(alpha=0.05)
x = Input(shape=(h, w, channel))
cnn_1 = Conv2D(nb_filter=64, nb_row=3, nb_col=3, border_mode='same', activation='relu', name='CNN1')(x)
cnn_1 = Conv2D(nb_filter=64, nb_row=3, nb_col=3, border_mode='same', activation='relu', name='CNN2')(cnn_1)
max_1 = MaxPooling2D(pool_size=(2, 2), border_mode='same', name='max1')(cnn_1)#(?, 120, 160, 64) 
#pad_2 = ZeroPadding2D(padding=(0, 1))(max_1) #This is used in local detection

cnn_2 = Conv2D(nb_filter=32, nb_row=3, nb_col=3, border_mode='same', activation='relu', name='CNN3')(max_1)#pad_2  #(?, 120, 160, 32) 
cnn_2 = Conv2D(nb_filter=32, nb_row=3, nb_col=3, border_mode='same', activation='relu', name='CNN4')(cnn_2)
max_2 = MaxPooling2D(pool_size=(2, 2), border_mode='same', name='max2')(cnn_2)#(?, 60, 80, 32) 


cnn_3 = Conv2D(nb_filter=16, nb_row=3, nb_col=3, border_mode='same', activation='relu', name='CNN5')(max_2)#(?, 60, 80, 16)
cnn_3 = Conv2D(nb_filter=16, nb_row=3, nb_col=3, border_mode='same', activation='relu', name='CNN6')(cnn_3)
max_3 = MaxPooling2D(pool_size=(2, 2), border_mode='valid', name='max3')(cnn_3)#(?, 30, 40, 16)


cnn_4 = Conv2D(nb_filter=8, nb_row=3, nb_col=3, border_mode='same', activation='relu', name='CNN7')(max_3)
cnn_4 = Conv2D(nb_filter=8, nb_row=3, nb_col=3, border_mode='same', activation='relu', name='CNN8')(cnn_4)#(?, 30, 40, 8) 

flat = Flatten()(cnn_4)
h_encoded = Dense(hidden_dim1, W_regularizer=l2(decay), b_regularizer=l2(decay),  activation='relu', name='Dense1_vae')(flat)  #use_bias=use_bias,
h_encoded = Dense(hidden_dim, W_regularizer=l2(decay), b_regularizer=l2(decay), activation='relu', name='Dense2_vae')(h_encoded)
z_mean = Dense(m_ori, W_regularizer=l2(decay), b_regularizer=l2(decay), name='Dense_mean_vae')(h_encoded)
z_log_var = Dense(m_ori, W_regularizer=l2(decay), b_regularizer=l2(decay), name='Dense_sigma_vae')(h_encoded)

## Sampler
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal_variable(shape=(m_ori, ), mean=0.,
                                       scale=epsilon_std)
    return z_mean + K.exp(z_log_var / 2) * epsilon

z = Lambda(sampling, output_shape=(m_ori,), name='Lambda')([z_mean, z_log_var])#(?, 6)

# Initialize
decoder_h1 = Dense(hidden_dim, W_regularizer=l2(decay), b_regularizer=l2(decay), activation='relu', name='Dense3_vae')
decoder_h = Dense(hidden_dim1, W_regularizer=l2(decay), b_regularizer=l2(decay), activation='relu', name='Dense4_vae')
decoder_mean = Dense(n, W_regularizer=l2(decay), b_regularizer=l2(decay),  activation='tanh', name='Dense5_vae')

## Decoder
h_decoded1 = decoder_h1(z)
h_decoded = decoder_h(h_decoded1)
x_hat = decoder_mean(h_decoded)


def dense2conv(args):
    x_hat = args
    return K.reshape(x_hat, (-1, hidden_h, hidden_w, hidden_c))

dense_out = Lambda(dense2conv, name='Lambda2')(x_hat)#(?, 30, 40, 8)


mer_5 = merge([Conv2D(16, 2, 2, activation='relu', border_mode='same', name='mer5_CNN')(UpSampling2D(size=(2, 2), name='upsam_5')(dense_out)), cnn_3], 
		 mode='concat', concat_axis=3, name='mer_5')#(?, 60, 80, 32)
cnn_5 = Conv2D(32, 3, 3, activation='relu', border_mode='same', name='cnn5')(mer_5)
cnn_5 = Conv2D(32, 3, 3, activation='relu', border_mode='same', name='cnn5.2')(cnn_5)

mer_6 = merge([Conv2D(32, 2, 2, activation='relu', border_mode='same', name='mer6_CNN')(UpSampling2D(size=(2, 2), name='upsam_6')(cnn_5)), cnn_2],
         mode='concat', concat_axis=3, name='mer_6')#(?, 120, 160, 64) 
cnn_6 = Conv2D(64, 3, 3, activation='relu', border_mode='same', name='cnn6')(mer_6)
cnn_6 = Conv2D(64, 3, 3, activation='relu', border_mode='same', name='cnn6.2')(cnn_6)

mer_7 = merge([Conv2D(64, 2, 2, activation='relu', border_mode='same', name='mer7_CNN')(UpSampling2D(size=(2, 2), name='upsam_7')(cnn_6)), cnn_1],
            mode='concat', concat_axis=3, name='mer_7')#(?, 240, 320, 128)
cnn_7 = Conv2D(128, 3, 3, activation='relu', border_mode='same', name='cnn7')(mer_7)
cnn_7 = Conv2D(128, 3, 3, activation='relu', border_mode='same', name='cnn7.2')(cnn_7)

conv10 = Conv2D(channel, 1, 1, activation='relu', name='cnn8')(cnn_7)#(?, 240, 320, 1) 

## loss
def vae_loss(x, conv10):
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    xent_loss = objectives.binary_crossentropy(x, conv10)
    mse_loss = K.mean(K.square(x - conv10), axis=[1, 2, 3])
    mse_kl_loss = mse_loss + kl_loss
    if use_loss == 'xent':
        return xent_loss + kl_loss
    elif use_loss == 'mse':
        return mse_kl_loss


vae = Model(x, conv10)
vae.summary()

adam = Adam(lr=0.0001)
vae.compile(optimizer=adam, loss=vae_loss)
##------------------------------------------------------------------------------------------------------------

checkpoint = ModelCheckpoint("/content/drive/My Drive/weight/loss_{loss_ty}".format(loss_ty=use_loss)+"_weight_{epoch:02d}-{val_loss:.15f}-{loss:.15f}.hdf5", verbose=0, save_best_only=True)

early_check = EarlyStopping(monitor='val_loss', patience=15, verbose=0, mode='min')

board_check = TensorBoard(log_dir='./tmp/log')


#vae.load_weights(weights, by_name=True)

##-----------------------------------------------------------------------------------------------------------------



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(activation="relu", name="CNN1", kernel_size=(3, 3), filters=64, padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(activation="relu", name="CNN2", kernel_size=(3, 3), filters=64, padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), name="max1", padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(activation="relu", name="CNN3", kernel_size=(3, 3), filters=32, padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(activation="relu", name="CNN4", kerne

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 240, 320, 3)   0                                            
____________________________________________________________________________________________________
CNN1 (Conv2D)                    (None, 240, 320, 64)  1792        input_1[0][0]                    
____________________________________________________________________________________________________
CNN2 (Conv2D)                    (None, 240, 320, 64)  36928       CNN1[0][0]                       
____________________________________________________________________________________________________
max1 (MaxPooling2D)              (None, 120, 160, 64)  0           CNN2[0][0]                       
___________________________________________________________________________________________

In [18]:
weights="loss_mse_weight_245-0.000000886071271-0.000000897372071.hdf5"
vae.load_weights("/content/drive/My Drive/weight/"+weights)
print(vae_loss)
print(vae.metrics_names)

<function vae_loss at 0x7ff52290ba60>
['loss']


In [0]:
import glob
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
#from goto import with_goto

def get_dynamic_image(frames, normalized=True):
    """ Takes a list of frames and returns either a raw or normalized dynamic image."""
    num_channels = frames[0].shape[2] 
    # to find number of components : A R G B. shape gives the details of the image returns 3 here in avenue dataset
    channel_frames = _get_channel_frames(frames, num_channels) 
    # splits the frame by channels
    channel_dynamic_images = [_compute_dynamic_image(channel) for channel in channel_frames]

    dynamic_image = cv2.merge(tuple(channel_dynamic_images)) 
    # opposite of split. merges the different channels
    if normalized:
        dynamic_image = cv2.normalize(dynamic_image, None, 0, 255, norm_type=cv2.NORM_MINMAX)
        dynamic_image = dynamic_image.astype('uint8')

    return dynamic_image


def _get_channel_frames(iter_frames, num_channels):
    """ Takes a list of frames and returns a list of frame lists split by channel. """
    frames = [[] for channel in range(num_channels)] 
    #here 3 []

    for frame in iter_frames:
        for channel_frames, channel in zip(frames, cv2.split(frame)): 
	    # zip used for mapping. example (1,2,3,4) and (4,5,6,7) to [(1,4),(2,5),(3,6),(4,7)]
            channel_frames.append(channel.reshape((*channel.shape[0:2], 1)))
    for i in range(len(frames)):
        frames[i] = np.array(frames[i])
    return frames


def _compute_dynamic_image(frames):
    """ For computing dynamic image"""
    num_frames, h, w, depth = frames.shape# number =20

    # Compute the coefficients for the frames.
    coefficients = np.zeros(num_frames)
    #print(num_frames)
    for n in range(num_frames):
        cumulative_indices = np.array(range(n, num_frames)) + 1
        coefficients[n] = np.sum(((2*cumulative_indices) - num_frames-1) / cumulative_indices) #1/t B_t

    # Multiply frames by the coefficients and sum the result.
    x1 = np.expand_dims(frames, axis=0) #v_t
    x2 = np.reshape(coefficients, (num_frames, 1, 1, 1))
    result = x1 * x2 #B_t * V_t
    return np.sum(result[0], axis=0).squeeze()



In [23]:

""" driver program for checking abnormality of dynamic image"""
pic=cv2.VideoCapture('/content/drive/My Drive/Testing Videos/05.avi') # 0 for taking pictures from camera
files = glob.glob ("/content/drive/My Drive/Colab Notebooks/dynamic_image.png") #For dynamic image position
break_flag = False 

#counter=0
# for exiting from the loop in case of end of the video
while(pic.isOpened()):
	X=[]
	i=0
	for i in range(20):
	        #  create frames from video source
		ret,frame=pic.read()
		if ret==False:
			break_flag = True
			break
			# goto .br
		cv2.imwrite(str(i)+'.jpg',frame)
		i+=1
	frames = glob.glob('*.jpg')
	frames = [cv2.imread(f) for f in frames]

	dyn_image = get_dynamic_image(frames, normalized=True) # to get dynamic image from the current number of extracted frames.
	cv2_imshow(frames[0])
	cv2_imshow(dyn_image)
	image=cv2.resize(dyn_image, (320,240), interpolation=cv2.INTER_CUBIC)
	X.append(image)
	X=np.array(X)
	np.max(X)
	X=X/np.max(X)
	print(X.shape)
	los=vae.evaluate(X,X)
	print(los)
	cv2.waitKey()
	if break_flag:
		break

#label .br
pic.release()
cv2.destroyAllWindows()

Output hidden; open in https://colab.research.google.com to view.